In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
import os
%matplotlib inline

/Users/Iskandar/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)
/Users/Iskandar/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


### Download , Extract and  Load the data

In [4]:
# Constants to be used ahead

data_path = "user_data" 
ssa_url = 'http://www.ssa.gov/oact/babynames/state/namesbystate.zip'
!pwd

/Users/Iskandar/Desktop/DataSciecneJobHunt/CapitalONe/c1 data science challenge


In [5]:
def download_extract_load(data_path,web_url):
    '''
    Downloads and extracts data and then calls read_in_babynames() function to load the data to a dataframe
    INPUT:
    data_path - path where data needs to be downlaoded. By default works in the current working directory
    web_url - url path to the babynames zipfile
    
    OUTPUT:
    
    data -  data frame with data loaded
    '''
    # creates path if it does not exist
    if not os.path.isdir(data_path): 
        os.makedirs(data_path)
    
    #Change directory to data path
    os.chdir(data_path)
    
    #Download Data if not already downloaded
    if not os.path.isfile("names.zip"):
        print "Downloading."
        import urllib
        urllib.urlretrieve(ssa_url, 'names.zip')
    
    else: print "Data already downloaded."
    print"Downloading complete."    
    
    if not os.path.isfile("AL.txt") or not os.path.isfile("WY.txt"):
        print "Extracting."
        import zipfile
        with zipfile.ZipFile('names.zip') as zf:
            #for member in zf.infolist():#Alternate method
                #zf.extract(member)
            zf.extractall() 
        
    else: print "Data already extracted."
    print"Extraction complete!!"
    
    data = read_in_babynames()
    return data
    

In [6]:
def read_in_babynames():
    """
    Reads in data from a given location
    INPUT:
    None
    
    OUTPUT:
    data- dataframe containing the contents of all the files
    
    """
    
    
    
    # List of all state(+DC = 51 in total) names to be used for reading in the files
    states = ['AL','AK','AZ','AR','CA','CO','CT','DC','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA',
    'ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA',
    'RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
    
    
    # Change working directory to the location of the files
    #os.chdir(path_to_files)
    
    # Initializing an empty dataframe to read in the data
    data = pd.DataFrame()
    counter = 0
    
    for state in states:
        counter+=1
        filename =  state +'.txt'
        #print filename as a check
        print "%d)%s loaded"%(counter,state)
        # The files dont have headers
        temp_data =  pd.read_csv(filename,sep = ",",header=None)
        data = data.append(temp_data)
    
    # Set column names
    data.columns = ["state","gender","year","name","state_frequency"]
    print "Data load is now complete. Do your thing!!"
    return data
    
  

In [8]:
data = download_extract_load(data_path,ssa_url)   

Downloading.
Extracting.
Extraction complete!!
1)AL loaded
2)AK loaded
3)AZ loaded
4)AR loaded
5)CA loaded
6)CO loaded
7)CT loaded
8)DC loaded
9)DE loaded
10)FL loaded
11)GA loaded
12)HI loaded
13)ID loaded
14)IL loaded
15)IN loaded
16)IA loaded
17)KS loaded
18)KY loaded
19)LA loaded
20)ME loaded
21)MD loaded
22)MA loaded
23)MI loaded
24)MN loaded
25)MS loaded
26)MO loaded
27)MT loaded
28)NE loaded
29)NV loaded
30)NH loaded
31)NJ loaded
32)NM loaded
33)NY loaded
34)NC loaded
35)ND loaded
36)OH loaded
37)OK loaded
38)OR loaded
39)PA loaded
40)RI loaded
41)SC loaded
42)SD loaded
43)TN loaded
44)TX loaded
45)UT loaded
46)VT loaded
47)VA loaded
48)WA loaded
49)WV loaded
50)WI loaded
51)WY loaded
Data load is now complete. Do your thing!!


## Descriptive analysis - THE QUERIES


In [9]:
# Look for missing data
data.isnull().sum(axis=0)

state              0
gender             0
year               0
name               0
state_frequency    0
dtype: int64

In [10]:
# Name the columns
data.columns = ["state","gender","year","name","state_frequency"]
data.head()

,state,gender,year,name,state_frequency
0,AL,F,1910,Mary,875
1,AL,F,1910,Annie,482
2,AL,F,1910,Willie,257
3,AL,F,1910,Mattie,232
4,AL,F,1910,Ruby,204


In [11]:
data.dtypes

state              object
gender             object
year                int64
name               object
state_frequency     int64
dtype: object

In [12]:
# CHanging the columns to string
data['state'] = map(lambda x: str(x),data['state'])
data['name'] = map(lambda x: str(x),data['name'])
data['gender'] = map(lambda x: str(x),data['gender'])

data.dtypes

state              object
gender             object
year                int64
name               object
state_frequency     int64
dtype: object

All things seem in order. Proceeding with Queries

### 1.  Please describe the format of the data files. Can you identify any limitations or distortions of the data?

The data is stored in tall format in ***comma seperated text files***, one for each state.
Each row has five values:
* **state**: the 2-letter code of the state.
* **gender**: one letter, M|F.
* **year**: the year of birth for this record, in 4-digit format.
* **name**: the name.
* **state_frequency**: an integer for the number of times this name occurred in that state in that year.

In the description provided by SSA, it is mentioned that only names which occur 5 or more times in an year are included
which introduces bias into the data. Hence this data is not a comprehensive representation of all baby names in America.
This bias also limits our ability to get an idea of population since we have no idea how many babies with names that 
occur less than 5 times exist in each state. I would therefore be careful about any generalizations from this data.

### 2.  What is the most popular name of all time? (Of either gender.)

In [13]:
# Top 10 names
(data.groupby(['gender','name']).sum().sort_values(by = ["state_frequency"],ascending = False))[:10]

year  state_frequency
gender name                              
M      James    10504600          4938965
       John     10506510          4829733
       Robert   10496851          4710600
       Michael   9932640          4295779
       William  10506510          3829026
F      Mary     10430202          3730856
M      David    10447162          3554102
       Richard  10427952          2529952
       Joseph   10489225          2479602
       Charles  10491166          2244617

##### The most popular name by itself is ***James***
We can see here that the most popular name for :

Women is **Mary** 

Men is **James**

### 3.  What is the most gender ambiguous name in 2013? 1945?

In [14]:
def find_common_names(year, data):
    
    '''
    Returns common names for males and females for a given year
    
    INPUT:
    year - year for which this needs doing
    data - data
    
    OUTPUT:
    names -  set with unisex names
    '''
    df = data[data['year'] == year]
    m_names = set(df[df['gender']=='M']['name'])
    f_names = set(df[df['gender']=='F']['name'])
    
    common_names =  m_names.intersection(f_names)
    return common_names

In [15]:
# Finding the names that appear in both male and female 
common_names_2013 =  find_common_names(2013,data)
common_names_1945 =  find_common_names(1945,data)

#### Define Gender ambiguity:
At this point we need to devise a metric to measure gender-ambiguity 
We look at it as the *name that has the similar percent as males as females asssociated with it:
* per_f = percentage of times the name appears as female
* per_m = percentage of times the name appears as male

**diff = per_f-per_m ~ 0**

Since per_f = 1- per_m  so **difference** can be reworked as ***abs(2*perf_m-1)***.

We do the reworking because it prevents us from having to use and extra self join in the future
when we need to calculate the difference


In [16]:
def gender_ambi(data, year):
    '''
    function to give the most data ambiguous names
    
    INPUT:
    data -  your data
    year -  year for which this operation names to be performed
    OUTPUT:
     
    list_of_ga_names - list of most gender ambiguous names
    
     
    '''
    
    # Get the set of gender ambiguous names
    ga_names = find_common_names(year,data)
    
    # Subset data to get a dataframe with gender ambiguous names for that year.
    df = data[data['year'] == year]
    df_ga = df[(df['name'].isin(ga_names))]
    
    # Calculating the groupby and sum. Reset index ensures we can use the groupby indexes as columns 
    #and access them easily for future funtions. It does the following 3 things
    #1) groups by given columns and removes years as a possible column by choosing stat_frequency 
    #2) then sums the groupby values
    #3) Resets index 
    df = df_ga.groupby(['name','gender'])['state_frequency'].sum().reset_index()
    
    # We calculate a second dataframe to count the total number of occurances for each name. Will be used to calculate
    # the percent male/female for each name
    tot_names = df_ga.groupby(['name']).sum().reset_index()
    
    
    # Merging the two using left join so I have the values aligned to calcuate the percent
    df_vals = pd.merge(df,tot_names,on='name',how='left')
    
    # Removing all the rows with gender = "F" as not needed going forward based on gender ambiguity formula
    df_final = df_vals[df_vals['gender'] == "M"]
    
    # Calculating the percent of male names and then the final difference using the formulae described above
    df_final['per_m'] = df_final['state_frequency_x']/df_final['state_frequency_y']
    df_final['diff'] = abs(2*df_final['per_m']-1)
    
    # Sort the final data frame by 
    ga = df_final.sort_values(by = ["diff"]).reset_index(True)
    min_diff = ga['diff'].min()
    
    # deriving the final list of names by finding the minimum difference
    solns = ga[ga['diff'] == min_diff]
    list_of_ga_names = solns['name']
    
    return list_of_ga_names

In [17]:
gender_ambi(data,2013)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    Nikita
1    Devine
2     Arlin
3     Sonam
4      Cree
Name: name, dtype: object

In [18]:
gender_ambi(data,1945)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    Maxie
Name: name, dtype: object

**Maxie** is the most gender ambiguous name for 1945 while 
for 2013 we have 
**Nikita, Devine, Cree, Sonam and Arlin**



### 4.  Of the names represented in the data, find the name that has had the largest percentage increase in popularity since 1980. Largest decrease?

### 5.  Can you identify names that may have had an even larger increase or decrease in popularity?

The following function solves for both of the above. We can use the Boolean consider_all to get the answers

In [19]:
data.head()

,state,gender,year,name,state_frequency
0,AL,F,1910,Mary,875
1,AL,F,1910,Annie,482
2,AL,F,1910,Willie,257
3,AL,F,1910,Mattie,232
4,AL,F,1910,Ruby,204


In [20]:
def inc_dec(year1,year2,data, inc = True, consider_all=True):
    '''
    In this case we consider also the names that did not exist in 1980,2014
    
    INPUT:
    
    year1,year2 - Years we wish to compare. 
    data -  your data
    inc - Boolen to decide whether to calculate increase or decrease. By default we calculate the increase
    consider_all- boolean consider all the names including the ones that didnt exist in 1980
    
    OUTPUT:
    
    soln - dataframe containing with max inc/dec and the value of inc/dec
    
    '''
    # Subset the database for specified years
    df1 = data[data['year'] == year1].reset_index(True)
    df2 = data[data['year'] == year2].reset_index(True)
    
    # Calculating the total appearances of each name for both the years
    names_y1 = df1.groupby(['name']).sum()['state_frequency'].reset_index(True)
    names_y2 = df2.groupby(['name']).sum()['state_frequency'].reset_index(True)

    # COmbining the two to form a master dataframe with 3 columns:
    #1) name
    #2) y1_freq 
    #3) y2_freq

    df = pd.merge(names_y1,names_y2,how = 'outer',on= 'name')
    df.columns = ['name','y1_freq','y2_freq']
    

    
    # Fill any null values with 0
    df.fillna(0,inplace = True)
    
    # Adding Laplacian smooting. Because if the increase is from 0 we get inf percent. So increasing all occurances by 1
    df['y1_freq'] +=1
    df['y2_freq'] +=1
    
    #Subsetting the data based on whether we're calculating increase
    if inc:
        # Get a subset where y2_freq (2014)>y1_freq(1980)
        
        dfinc = df[df['y2_freq']>df['y1_freq']]
        dfinc['diff'] = (dfinc['y2_freq'] - dfinc['y1_freq'])
        dfinc['perc'] = dfinc['diff'] /  dfinc['y1_freq']
        print 
        # Sort the final data frame by 
        dfinc = dfinc.sort_values(by = ["perc"] , ascending = False).reset_index(True)
        
        if not consider_all:
        # If we only wish to consider the names that existed in 1980. Since their value has been set to 1
            dfinc =  dfinc[dfinc['y1_freq'] > 1]
        
        max_inc = dfinc['perc'].max()
        
        soln =dfinc[dfinc["perc"] == max_inc][["name","perc"]]        
      
    #Subsetting the data based on whether we're calculating decrease
    else:     
        # Get a subset where y1_freq (1980)>y2_freq(2014)
        dfdec = df[df['y1_freq']>df['y2_freq']]
        dfdec['diff'] = (dfdec['y1_freq'] - dfdec['y2_freq'])
        dfdec['perc'] = dfdec['diff'] /  dfdec['y1_freq']
        
        # Sort the final data frame by 
        dfdec = dfdec.sort_values(by = ["perc"] , ascending = False).reset_index(True)
        max_dec = dfdec['perc'].max()
        soln =dfdec[dfdec["perc"] == max_dec][["name","perc"]]
        
    
    return soln

##### Largest decrease not considering all(only the names that existed in 1980)

In [21]:
inc_dec(1980,2014,data, inc= False, consider_all=False)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,perc
0,Tonya,0.999675


##### Largest increase not considering all (only the names that existed in 1980)

In [22]:
inc_dec(1980,2014,data, inc = True, consider_all=False)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,perc
114,Colton,1055.0


##### Largest decrease considering all (including the names that did not exist in 1980)

In [23]:
inc_dec(1980,2014,data, inc= False, consider_all=True)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,perc
0,Tonya,0.999675


##### Largest increase considering all (including the names that did not exist in 1980)

In [24]:
inc_dec(1980,2014,data, inc = True, consider_all=True)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,perc
0,Jayden,13419.0


Biggest increase considering only the names that existed in 1980 ***Colton*** 

Biggest decrease for both with all and without all  ***Tonya***

Biggest increase considering all the names ***Jayden***


# Appendix

In [157]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                           'two', 'two', 'one', 'three'],
                      'C' : np.random.randn(8),
                     'D' : np.random.randn(8)})

In [158]:
df

,A,B,C,D
0,foo,one,-0.251833,-1.102170
1,bar,one,0.116685,0.775409
2,foo,two,-1.386834,1.246479
3,bar,three,2.215657,-0.810762
4,foo,two,0.046645,-0.406791
5,bar,two,0.989874,-2.178078
6,foo,one,-1.334310,0.888156
7,foo,three,0.538030,0.542912


In [174]:
grouped = df.groupby(['A','B'])
grouped.count()

C  D
A   B          
bar one    1  1
    three  1  1
    two    1  1
foo one    2  2
    three  1  1
    two    2  2

In [309]:
def inc_dec_max(year1,year2,data, inc = True, consider_all =True):
    '''
    In this case we consider also the names that did not exist in 1980,2014
    
    INPUT:
    
    year1,year2 - Years we wish to compare. 
    data -  your data
    inc - Boolen to decide whether to calculate increase or decrease. By default we calculate the increase
    
    OUTPUT:
    
    soln - dataframe containing with max inc/dec and the value of inc/dec
    
    '''
    # Subset the database for specified years
    df1 = data[data['year'] == year1].reset_index(True)
    df2 = data[data['year'] == year2].reset_index(True)
    
    # Calculating the total appearances of each name for both the years
    names_y1 = df1.groupby(['name']).sum()['state_frequency'].reset_index(True)
    names_y2 = df2.groupby(['name']).sum()['state_frequency'].reset_index(True)

    # COmbining the two to form a master dataframe with 3 columns:
    #1) name
    #2) y1_freq 
    #3) y2_freq

    df = pd.merge(names_y1,names_y2,how = 'outer',on= 'name')
    df.columns = ['name','y1_freq','y2_freq']
    
    if not consider_all:
        df = df[ ~(df['y1_freq'].isnull())]
       
      
    # Fill any null values with 0
    df.fillna(0,inplace = True)

    # Adding Laplacian smooting. Because if the increase is from 0 we get inf percent. So increasing all occurances by 1
    df['y1_freq'] +=1
    df['y2_freq'] +=1

    
    #Subsetting the data based on whether we're calculating increase or decrease
    if inc:
        
        # Get a subset where y2_freq (2014)>y1_freq(1980)
        dfinc = df[df['y2_freq']>df['y1_freq']]
        dfinc['diff'] = (dfinc['y2_freq'] - dfinc['y1_freq'])
        dfinc['perc'] = dfinc['diff'] /  dfinc['y1_freq']
        
        # Sort the final data frame by 
        dfinc = dfinc.sort_values(by = ["perc"] , ascending = False).reset_index(True)
        max_inc = dfinc['perc'].max()
        soln =dfinc[dfinc["perc"] == max_inc][["name","perc"]]
        
        
    else:     
        
        # Get a subset where y1_freq (1980)>y2_freq(2014)
        dfdec = df[df['y1_freq']>df['y2_freq']]
        dfdec['diff'] = (dfdec['y1_freq'] - dfdec['y2_freq'])
        dfdec['perc'] = dfdec['diff'] /  dfdec['y1_freq']
        
        # Sort the final data frame by 
        dfdec = dfdec.sort_values(by = ["perc"] , ascending = False).reset_index(True)
        max_dec = dfdec['perc'].max()
        soln =dfdec[dfdec["perc"] == max_dec][["name","perc"]]
    return soln

In [311]:
inc_dec_max(1980,201,data,False,False)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,name,perc
0,Jill,0.99978


In [305]:
a,b=inc_dec_max(1980,2013,data,True,False)

/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Iskandar/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [306]:
a

,name,perc
0,Colton,1075.833333


In [307]:

b

,index,name,y1_freq,y2_freq,diff,perc
0,1061,Colton,6.0,6461.0,6455.0,1075.833333
1,382,Aria,6.0,5117.0,5111.0,851.833333
2,2195,Isabella,24.0,17574.0,17550.0,731.250000
3,5040,Skylar,6.0,4225.0,4219.0,703.166667
4,1077,Cooper,8.0,4911.0,4903.0,612.875000
5,3806,Mila,6.0,3673.0,3667.0,611.166667
6,3675,Mateo,6.0,3557.0,3551.0,591.833333
7,4763,Serenity,8.0,4423.0,4415.0,551.875000
8,3488,Makayla,7.0,3255.0,3248.0,464.000000
9,538,Bella,9.0,4155.0,4146.0,460.666667
